In [ ]:
import sys
import os
file_path = os.path.abspath("/Users/noemieclaret/Desktop/")
sys.path.append(file_path)
from beyond_gd_starter import *;
from mnist_experiments import FitNN
from mnist_experiments import FitNN

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset


from scipy.stats import ttest_ind

In [ ]:

experiments = [
    ("mlp", "adam"),
    ("mlp", "adagrad"),
    ("mlp", "truncatedsgd"),
    ("mlp", "truncatedadagrad"),
    ("conv", "adam"),
    ("conv", "adagrad"),
    ("conv", "truncatedsgd"),
    ("conv", "truncatedadagrad"),
]

for model_type, optimizer in experiments:
    print(f"\nRunning experiment: Model={model_type.upper()}, Optimizer={optimizer.upper()}...\n")
    FitNN(model_type=model_type, optimizer_type=optimizer, learning_rate=1e-2)


In [20]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, TensorDataset
import beyond_gd as BGD
from mnist_experiments import FitNN
from scipy.stats import ttest_ind

# Step sizes to compare
alpha_1, alpha_2 = 1e-2, 1e-4
num_trials = 5  # Monte Carlo trials per sample
num_samples = 10  # Number of times to repeat Monte Carlo trials

# Run experiments on convolutional network
experiments = [
    ("conv", "adam"),
    ("conv", "truncatedadagrad")
]

final_vars = {exp: [] for exp in experiments}

for _ in range(num_samples):
    temp_vars = {exp: [] for exp in experiments}
    
    for model_type, optimizer in experiments:
        for _ in range(num_trials):
            loss = FitNN(model_type=model_type, optimizer_type=optimizer, learning_rate=alpha_1)
            temp_vars[(model_type, optimizer)].append(loss)
    
    for key in temp_vars:
        final_vars[key].append(np.var(temp_vars[key]))

var_adagrad_alpha_1 = final_vars[("conv", "truncatedadagrad")]
var_adam_alpha_1 = final_vars[("conv", "adam")]

final_vars = {exp: [] for exp in experiments}

for _ in range(num_samples):
    temp_vars = {exp: [] for exp in experiments}
    
    for model_type, optimizer in experiments:
        for _ in range(num_trials):
            loss = FitNN(model_type=model_type, optimizer_type=optimizer, learning_rate=alpha_2)
            temp_vars[(model_type, optimizer)].append(loss)
    
    for key in temp_vars:
        final_vars[key].append(np.var(temp_vars[key]))

var_adagrad_alpha_2 = final_vars[("conv", "truncatedadagrad")]
var_adam_alpha_2 = final_vars[("conv", "adam")]


Setting backend device to use mps
Using standard Adam optimizer
Epoch [1 / 5]. Mean loss: 0.2439
Time for epoch 1: 13.57 seconds
Test accuracy: 95.49%

Epoch [2 / 5]. Mean loss: 0.1131
Time for epoch 2: 13.40 seconds
Test accuracy: 97.28%

Epoch [3 / 5]. Mean loss: 0.1046
Time for epoch 3: 14.31 seconds
Test accuracy: 94.95%

Epoch [4 / 5]. Mean loss: 0.1009
Time for epoch 4: 14.17 seconds
Test accuracy: 96.84%

Epoch [5 / 5]. Mean loss: 0.0948
Time for epoch 5: 15.24 seconds
Test accuracy: 96.61%

Setting backend device to use mps
Using standard Adam optimizer
Epoch [1 / 5]. Mean loss: 0.2997
Time for epoch 1: 14.88 seconds
Test accuracy: 96.45%

Epoch [2 / 5]. Mean loss: 0.1047
Time for epoch 2: 15.01 seconds
Test accuracy: 97.77%

Epoch [3 / 5]. Mean loss: 0.0936
Time for epoch 3: 14.74 seconds
Test accuracy: 97.34%

Epoch [4 / 5]. Mean loss: 0.0854
Time for epoch 4: 14.84 seconds
Test accuracy: 97.23%

Epoch [5 / 5]. Mean loss: 0.0852
Time for epoch 5: 14.76 seconds
Test accuracy: 

KeyboardInterrupt: 

In [ ]:

# Compute robustness ratios
R_adagrad = np.array(var_adagrad_alpha_1)/ np.array(var_adagrad_alpha_2)
R_adam = np.array(var_adam_alpha_1) / np.array(var_adam_alpha_2)

# Hypothesis testing
statistic, p_value = ttest_ind(R_adagrad, R_adam, alternative='less')

# Print results
print(f"Variance ratios for Truncated AdaGrad: {R_adagrad}")
print(f"Variance ratios for Adam: {R_adam}")
print(f"T-statistic: {statistic}")
print(f"P-value: {p_value}")
if p_value < 0.05:
    print("Reject the null hypothesis: There is no statistical evidence that Adam is as robust as Truncated AdaGrad \n (against alternative: Truncated Adagrad is more robust).")
else:
    print("Fail to reject the null hypothesis: Truncated AdaGrad is at least as robust as Adam.")


print(R_adagrad)
print(R_adam)


Variance ratios for Truncated AdaGrad: [4.12988869e-05 2.79532695e-02 5.84208251e-04 6.24049675e-04]
Variance ratios for Adam: [2.95174487e+05 4.96673772e+05 4.38463727e+01 2.91975017e+05]
T-statistic: -2.650998194410575
P-value: 0.018988962319497923
Reject the null hypothesis: There is no statistical evidence that Adam is as robust as Truncated AdaGrad 
 (against alternative: Truncated Adagrad is more robust).
[4.12988869e-05 2.79532695e-02 5.84208251e-04 6.24049675e-04]
[2.95174487e+05 4.96673772e+05 4.38463727e+01 2.91975017e+05]


In [ ]:
R_adagrad

array([4.12988869e-05, 2.79532695e-02, 5.84208251e-04, 6.24049675e-04])

In [11]:
R_adam

array([2.95174487e+05, 4.96673772e+05, 4.38463727e+01, 2.91975017e+05])